In [1]:

import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import networkx
#from textblob import TextBlob
import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer


sns.set(font_scale=1.5)
sns.set_style("whitegrid")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/kaushik/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [88]:
Consumer_Key="7ULNGRHNK9zJIxjc5KLxuMTeZ"
Consumer_Secret="1jPRtvBFxvZwUzd9Gk9gUvYSMnLHrBEQrcVieQjGeEDqSPnnTf"
Access_Token="1458418334081425410-GiOCzDRZDkCVCOxPgVcpBSv7454OQV"
Access_Secret="k58HmCgCHH7cfjChhMJyhJSHeZ0UGo6f0MjYNxR0H1gt6"

In [89]:
auth = tw.OAuthHandler(Consumer_Key,Consumer_Secret)
auth.set_access_token(Access_Token,Access_Secret)
api = tw.API(auth, wait_on_rate_limit = True)


In [90]:
search_food ="Food OR food OR Foodie OR Delicious OR Breakfast OR Lunch OR Dinner -filter:retweets"
food_tweets = tw.Cursor(api.search_tweets,
                        q = search_food,
                        lang="en",
                        tweet_mode="extended"
                        ).items(5000)
food_tweets_list = [[tweet.created_at, tweet.place, tweet.user.name, tweet.full_text] for tweet in food_tweets]
for i in range(len(food_tweets_list)):
    print()
    print(food_tweets_list[i])
    print()

Rate limit reached. Sleeping for: 828


KeyboardInterrupt: 

In [ ]:
tweets_df = pd.DataFrame(food_tweets_list)
#rename the columns
tweets_df.columns = ['Created at', 'Place', 'User', 'Text']

In [52]:
tweets_df.head()

,Created at,Place,User,Text
0,2021-11-10 15:29:57+00:00,None,MESD School Health Services,Reynolds Middle School and David Douglas High ...
1,2021-11-10 15:29:57+00:00,None,jack! @ wwdits,I prommy to corral all my spoilers in a singul...
2,2021-11-10 15:29:57+00:00,None,Thorbaby65🇺🇲,@Maithedane One family dinner at a time please...
3,2021-11-10 15:29:57+00:00,None,ResDiary,Was wonderful to get a mention in this @IrishT...
4,2021-11-10 15:29:56+00:00,None,Garrick Kebede aka daytradernupe,@ClayTravis Here is food inflation. https://t....


In [55]:
#save the dataframe as a csv
tweets_df.to_csv("/Users/kaushik/data management with python/Twitter_Sentiment_Analysis/food_tweets.csv", index = False)

In [59]:
tweets_df = pd.read_csv("/Users/kaushik/data management with python/Twitter_Sentiment_Analysis/food_tweets.csv")

In [60]:
tweets_df.head()

,Created at,Place,User,Text
0,2021-11-10 15:29:57+00:00,NaN,MESD School Health Services,Reynolds Middle School and David Douglas High ...
1,2021-11-10 15:29:57+00:00,NaN,jack! @ wwdits,I prommy to corral all my spoilers in a singul...
2,2021-11-10 15:29:57+00:00,NaN,Thorbaby65🇺🇲,@Maithedane One family dinner at a time please...
3,2021-11-10 15:29:57+00:00,NaN,ResDiary,Was wonderful to get a mention in this @IrishT...
4,2021-11-10 15:29:56+00:00,NaN,Garrick Kebede aka daytradernupe,@ClayTravis Here is food inflation. https://t....


In [65]:
def clean_tweet(txt):
    txt = re.sub(r'@[A-Za-z0-9_]+', '', txt) # Keeps text A-Z, a-z, 0-9
    #txt = re.sub(r'#[A-Z0-9]+', '', txt)
    txt = re.sub(r'RT : ', '', txt)
    txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', txt) # remove url
    txt = re.sub(r'&amp;', '', txt)
    txt = re.sub(r'ðŸ™', '', txt)
    txt = re.sub(r'\n', ' ', txt) 
    return txt
tweets_df['Cleaned_Text'] = tweets_df['Text'].apply(clean_tweet)

In [62]:
tweets_df

,Created at,Place,User,Text,Cleaned_Text
0,2021-11-10 15:29:57+00:00,NaN,MESD School Health Services,Reynolds Middle School and David Douglas High ...,Reynolds Middle School and David Douglas High ...
1,2021-11-10 15:29:57+00:00,NaN,jack! @ wwdits,I prommy to corral all my spoilers in a singul...,I prommy to corral all my spoilers in a singul...
2,2021-11-10 15:29:57+00:00,NaN,Thorbaby65🇺🇲,@Maithedane One family dinner at a time please...,One family dinner at a time please..... 😒
3,2021-11-10 15:29:57+00:00,NaN,ResDiary,Was wonderful to get a mention in this @IrishT...,Was wonderful to get a mention in this articl...
4,2021-11-10 15:29:56+00:00,NaN,Garrick Kebede aka daytradernupe,@ClayTravis Here is food inflation. https://t....,Here is food inflation.
...,...,...,...,...,...
4995,2021-11-10 15:04:45+00:00,NaN,Otte Blair MS,"OBMS students raised $1,415.03 during the blac...","OBMS students raised $1,415.03 during the blac..."
4996,2021-11-10 15:04:45+00:00,NaN,Marcus,@wasserpest Your monthly pocket money is $100....,Your monthly pocket money is $100. Your cloth...
4997,2021-11-10 15:04:44+00:00,NaN,REBECCA PROFFITT,"@Venmo I need tacos, or wings, or any kind of ...","I need tacos, or wings, or any kind of food f..."
4998,2021-11-10 15:04:43+00:00,NaN,Rural Policy Group,"@MikeSTMack Join us to investigate how food, f...","Join us to investigate how food, farming and ..."


In [50]:
for index, row in tweets_df['Text'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    if score['neg']>score['pos']:
        tweets_df.loc[index, "Sentiment"] = "negative"
    elif score['pos']>score['neg']:
        tweets_df.loc[index, "Sentiment"]="positive"
    else:
        tweets_df.loc[index,"Sentiment"]="neutral"
        tweets_df.loc[index, 'neg'] = score['neg']
        tweets_df.loc[index, 'neu'] = score['neu']
        tweets_df.loc[index, 'pos'] = score['pos']
        tweets_df.loc[index, 'compound'] = score['compound']

In [74]:
tweets_df['Cleaned_Text'].iloc[6]

' Lol no airline miles. Braised bird for dinner'

In [75]:
tweets_df['Text'].iloc[6]

'@Lukewearechange Lol no airline miles. Braised bird for dinner'